## Azure Quantum Q# notebook

To use Jupyter Notebooks with Q#, the `qsharp-lang` and `jupyterlab` Python packages should be installed.

To render charts as shown in this notebook, Matplotlib with notebook widgets support should be installed via `ipympl`.

To submit to Azure Quantum as shown in this notebook, the `azure-quantum` package needs to be installed.

You can install all the above packages in your Python environment by running the below in your terminal:

```bash
pip install jupyterlab qsharp-lang ipympl azure-quantum
```

In [ ]:
# Run this cell first to enable the `%%qsharp` magic command used in later cells
import qsharp


The below cell shows Q# being written directly inside a Jupyter Notebook using the `%%qsharp` 'magic' command

In [ ]:
%%qsharp

// This makes the DumpMachine() function available.
open Microsoft.Quantum.Diagnostics;

operation RandomBit() : Result {
    // Qubits are only accesible for the duration of the scope where they
    // are allocated and are automatically released at the end of the scope.
    use qubit = Qubit();

    // Set the qubit in superposition by applying a Hadamard transformation.
    H(qubit);

    DumpMachine();

    // Measure the qubit. There is a 50% probability of measuring either
    // `Zero` or `One`.
    let result = M(qubit);

    Message($"The result of the measurement is {result}.");

    // Reset the qubit so it can be safely released.
    Reset(qubit);
    return result;
}

// Call the operation we defined above.
RandomBit();


As well as invoking Q# operations immediately inline (as the last line in the cell above does), the defined operations can also be invoked from Python

In [ ]:
r = qsharp.eval("RandomBit()")

if r == qsharp.Result.One:
    i = 1
else:
    i = 0

print(f"Python: Got integer {i}!")


You can use other Python libraries, such as Matplotlib, to run the Q# code for a number of 'shots' and visualize the results

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

results = qsharp.run("RandomBit()", shots=1000)
# Sort the results so that the histogram labels appear in the correct order
results.sort()
# Count the number of times each result appears
counts = Counter(results)

(values, counts) = counts.keys(), counts.values()
xlabels = np.arange(len(counts))
plt.title("RandomBit() Results")
plt.bar(xlabels, counts)
plt.xticks(xlabels, values)
plt.show()


## Submitting jobs to Azure Quantum

Different quantum hardware supports different capabilities, but all Azure Quantum providers support the 'base profile'
as defined in the 'Quantum Intermediate Representation' (QIR) specification. (For more details see <https://aka.ms/qdk.qir>)

To develop code using this base profile, reintialize the Q# compiler, connect to your Azure Quantum workspace, and submit the job.

__NOTE__: The workspace connection must be authenticated. If running this notebook locally, it will automatically open
a browser to sign in. However if running remotely, such as in a GitHub Codespace or Docker container, you may have to
sign-in by installing the Azure CLI and running `az login --use-device-code` before the connection will succeed.

In [ ]:
import azure.quantum

# Reset the compiler to target the 'base profile' that all hardware supports
qsharp.init(target_profile=qsharp.TargetProfile.Base)

# Ensure the below reflect the Azure Quantum workspace you'd like to use
subscription_id = 'MY_SUBSCRIPTION_ID'
resource_group = 'MY_RESOURCE_GROUP'
workspace_name = 'MY_WORKSPACE_NAME'
location = 'MY_LOCATION'

# Connect to an Azure Quantum workspace
workspace = azure.quantum.Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    name=workspace_name,
    location=location,
)


In [ ]:
%%qsharp

operation Random() : Result {
    use q = Qubit();
    H(q);
    let result = M(q);
    Reset(q);
    return result
}

operation RandomNBits(N: Int): Result[] {
    mutable results = [];
    for i in 0 .. N - 1 {
        let r = Random();
        set results += [r];
    }
    return results
}


In [ ]:
# Run the above code for 100 shots against the Rigetti simulator

operation = qsharp.compile("RandomNBits(4)")
target = workspace.get_targets("rigetti.sim.qvm")
job = target.submit(operation, "my-azure-quantum-job", input_params={ "count": 100 })

# Wait for the job to complete
job.get_results()
